In [4]:
# Standard definitions needed for script
import numpy as np
import matplotlib.pyplot as plt
import numpy.ma as ma
import math
from astropy.table import Table
from astropy.io import fits
import astropy.units as u
from astropy.coordinates.sky_coordinate import SkyCoord
from astropy.units import Quantity
from numpy.polynomial.polynomial import polyfit

In [2]:
# set-up for query to GAIA archive
from astroquery.gaia import Gaia
tables = Gaia.load_tables(only_names=True)
for table in (tables):
     print (table.get_qualified_name())

ModuleNotFoundError: No module named 'astroquery'

In [ ]:
# send request to GAIA archive
# the items in the box command are (corrdinate system, RA in deg, Dec in 
deg, box width in RA and Dec in deg)
query = "SELECT * \
FROM gaiadr2.gaia_source AS g, gaiadr2.tmass_best_neighbour AS tbest, 
gaiadr1.tmass_original_valid AS tmass \
WHERE g.source_id = tbest.source_id AND tbest.tmass_oid = tmass.tmass_oid \
AND CONTAINS(POINT('ICRS',g.ra,g.dec),BOX('ICRS',277.49,-1.00,5.,5.))=1 AND (parallax >=1.7 AND parallax_over_error >=3);"

In [ ]:
job1 = Gaia.launch_job_async(query, dump_to_file=False)
# this retrieves the result from the query above and puts into a table with the name "p"
# The next line prints out the labels for all of the columns in the table
pp = job1.get_results()
print (p.colnames)

In [ ]:
ra_big = pp["ra"]
dec_big = pp["dec"]
par_big = pp["parallax"]
pare_big = pp["parallax_error"]
parsn_big = pp["parallax_over_error"]
pmra_big = pp["pmra"]
pmdec_big = pp["pmdec"]
print(len(ra_big))
nn = 0
for i in range(0,len(ra_big)):
     dist = 1000/par_big[i]
     if(dist > 350. and dist <550. and parsn_big[i]>10.):
         nn = nn + 1
print("Number of sources in distance range:", nn)
secperyr = 60*60*24*365.25
to_kms = 443./1000. * 1.496e8/secperyr
pm_mag = np.zeros(0)
for i in range(0,len(ra_big)):
     dist = 1000/par_big[i]
     if(dist > 350. and dist <550. and parsn_big[i]>10.):
         pm2 = pmra_big[i]*pmra_big[i] + pmdec_big[i]*pmdec_big[i]
         pm = np.power(pm2,0.5)
         pm_mag = np.append(pm_mag,pm)
print(len(pm_mag))
plt.hist(pm_mag,bins=100,range=(0,100))
plt.xlabel("Magnitude of Proper Motion (km/sec)")
plt.ylabel("Number of sources with S/N>10 in Parallax")
plt.text(27,500,"Gaia sources in 5 deg radius of (18:30, -1 deg)")
plt.text(40,450,"With Parallax S/N > 10")
plt.text(40,400,"With 350 pc < Dist < 550 pc")
plt.savefig("big-Serp-pms.png")
plt.show()